In [1]:
import requests
import pandas as pd
import numpy as np
import time 
import json
import os
from datetime import datetime, timedelta

# REQUEST AND CLEAN API 

Practice with the API to understand the requests and results

In [2]:
# API requests: 
btc = requests.get("https://api.cryptowat.ch/markets/kraken/btcusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7")
#Jsonize
btc = btc.json()

# btc['result']['86400']

# Convert jsonized lists to DF 
btc = pd.DataFrame(btc['result']['86400'])

# Rename DF columns 
btc.columns = ['date', 'open_price', 'hi_price', 'lo_price', 'close_price', 'vol', 'quote_vol']

# Select only date and closing value 
btc = btc[['date','close_price']]

# Convert Epoch time (secs since 1970) into datetime object
btc['date'] = pd.to_datetime(btc['date'], unit='s')

#Select date range from 2016 - 2021
BTC = (btc.loc[btc["date"].between("2016-01-01", "2021-12-31")]).reset_index(drop=True)


# CLEAN API DATA FUNCTION

In [3]:
def clean_api_request(req): #Inouts a cryptowatch API request
    req = req.json()
    # Convert jsonized lists to DF 
    req = pd.DataFrame(req['result']['86400'])
    # Rename DF columns 
    req.columns = ['date', 'open_price', 'hi_price', 'lo_price', 'close_price', 'vol', 'quote_vol']
    # Select only date and closing value 
    req = req[['date','close_price']]
    # Convert Epoch time (secs since 1970) into datetime object
    req['date'] = pd.to_datetime(req['date'], unit='s')
    #Select date range from 2016 - 2021
    req = (req.loc[req["date"].between("2016-01-01", "2021-12-31")]).reset_index(drop=True)
    return req # returns a DF with date and daily close cryptocurrency price for every request

# CRYPTO API REQUESTS AND DATA SAVING

In [4]:
BTC = requests.get("https://api.cryptowat.ch/markets/kraken/btcusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7")
LTC = requests.get("https://api.cryptowat.ch/markets/kraken/ltcusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7")
ETH = requests.get("https://api.cryptowat.ch/markets/kraken/ethusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7") 
XRP = requests.get("https://api.cryptowat.ch/markets/kraken/xrpusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7") 
DOGE = requests.get("https://api.cryptowat.ch/markets/kraken/dogeusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7") 

XRP and Doge are missing some historiucal data when pulling the API data, there are no records there before 2017, may be becayse the value for those crypto  was too low.

XRP had only 1687 rows (days of trade, so missing before 2017ish) 

DOGE has onlky 743 rows 

I found historical for those 2 cryptos in Kaggle. 
https://www.kaggle.com/datasets/sudalairajkumar/cryptocurrencypricehistory?resource=download&select=coin_XRP.csv

In [5]:
# Pass the API results through the cleaning function
BTC = clean_api_request(BTC)
LTC = clean_api_request(LTC)
ETH = clean_api_request(ETH)
#XRP and DPOGE data pulled from Kaggle 
XRP = pd.read_csv("../data/XRP_p_16-21.csv")
DOGE = pd.read_csv("../data/dogecoin_p_16_21.csv")

In [6]:
# Change column names 
BTC = BTC.rename(columns={"close_price": "btc_p"})
LTC = LTC.rename(columns={"close_price": "ltc_p"})
ETH = ETH.rename(columns={"close_price": "eth_p"})

In [7]:
#save to csv
BTC.to_csv("../final_data/btc.csv")
LTC.to_csv("../final_data/ltc.csv")
ETH.to_csv("../final_data/eth.csv")
XRP.to_csv("../final_data/xrp.csv")
DOGE.to_csv("../final_data/doge.csv")

# COMODITIES 

In [9]:
OIL_BRENT = pd.read_csv("../data/brent-crude-oil-prices-16-21.csv")
OIL_WTI = pd.read_csv("../data/WTI_oil_prices_16-21.csv")
GOLD = pd.read_csv("../data/gold_prices_16-21.csv")
SILVER = pd.read_csv("../data/silver_prices_16-21.csv")
STEEL = pd.read_csv("../data/Steel_prices_16-21.csv")

com_list = [OIL_WTI, OIL_BRENT, GOLD, SILVER,STEEL]  # List of commodity dataframes

In [10]:
# We convert 'date' column into the index to merge later with other DFs
OIL_WTI = OIL_WTI.set_index('date')
OIL_BRENT = OIL_BRENT.set_index('date')
GOLD = GOLD.set_index('date')
SILVER = SILVER.set_index('date')
STEEL = STEEL.set_index('date')

In [11]:
# Merging DF
merge=pd.merge(OIL_WTI,OIL_BRENT, how='inner', left_index=True, right_index=True)
merge_2 = pd.merge(merge,GOLD, how='inner', left_index=True, right_index=True)
merge_3 = pd.merge(merge_2,SILVER, how='inner', left_index=True, right_index=True)
comm_df = pd.merge(merge_3,STEEL, how='inner', left_index=True, right_index=True)

# CURRENCIES (£-USD, EURO-USD, USD-Yuan, USD-yen)

In [12]:
gbp = pd.read_csv("../data/USD_GBP_16-21.csv")
eur = pd.read_csv("../data/USD_EUR_16-21.csv")
yua = pd.read_csv("../data/USD_CNY_16-21.csv")
yen = pd.read_csv("../data/USD_JPY_16-21.csv")

In [13]:
#Convert "date" column to desired format
gbp['date'] = pd.to_datetime(gbp['date'])
gbp['date'] = gbp['date'].dt.strftime('%d/%m/%Y')

eur['date'] = pd.to_datetime(eur['date'])
eur['date'] = eur['date'].dt.strftime('%d/%m/%Y')

yua['date'] = pd.to_datetime(yua['date'])
yua['date'] = yua['date'].dt.strftime('%d/%m/%Y')

yen['date'] = pd.to_datetime(yen['date'])
yen['date'] = yen['date'].dt.strftime('%d/%m/%Y')

In [14]:
# We convert 'date' column into the index to merge later with other DFs
gbp = gbp.set_index('date')
eur = eur.set_index('date')
yua = yua.set_index('date')
yen = yen.set_index('date')

currencies = [gbp,eur,yua,yen] # List of dataframes

In [15]:
# We concat the currencies in 1 DF
curr_df = pd.concat(currencies,axis=1,join="inner",ignore_index=False,sort=False, copy=False) 

# CURRENCIES AND COMM concat 

In order to have the same rows and dates to compare

In [16]:
# We merge currencies and commodities to get the same trade days
main_df = pd.merge(curr_df,comm_df,how="inner",on='date')

# INTEREST RATE INDEXES 

In [17]:
# Load the files
libor = pd.read_csv("../data/LIBOR_16-21.csv")
eonia = pd.read_csv("../data/eonia_16-21.csv")
fed = pd.read_csv("../data/fed-funds-rate-16-21.csv")

interest_list = [libor, eonia,fed]   # List of dataframes

In [18]:
# convert date columun to the desired format and set it as index 
libor['date'] = pd.to_datetime(libor['date'])
libor['date'] = libor['date'].dt.strftime('%d/%m/%Y')
libor = libor.set_index('date')

eonia['date'] = pd.to_datetime(eonia['date'])
eonia['date'] = eonia['date'].dt.strftime('%d/%m/%Y')
eonia = eonia.set_index('date')

fed['date'] = pd.to_datetime(fed['date'])
fed['date'] = fed['date'].dt.strftime('%d/%m/%Y')
fed = fed.set_index('date')

In [19]:
# We merge the DF
rates_1 =  pd.merge(libor,eonia,how="inner",on='date')
rates =  pd.merge(rates_1,fed,how="inner",on='date')

In [20]:
# Merge rates with main DF
main_=  pd.merge(main_df,rates,how="inner",on='date')

# Merge cryptos with main DF

In [43]:
# Load the files
BTC = pd.read_csv("../final_data/btc.csv")
LTC = pd.read_csv("../final_data/ltc.csv")
ETH = pd.read_csv("../final_data/eth.csv")
XRP = pd.read_csv("../final_data/xrp.csv")
DOGE = pd.read_csv("../final_data/doge.csv")

In [42]:
# # Rename some columns on XRP and DOGE
# XRP = XRP.rename(columns={'Date':'date'})
# DOGE = DOGE.rename(columns={'Date':'date'})

In [44]:
# Convert date column to desired data type:
BTC['date'] = pd.to_datetime(BTC['date'])
BTC['date'] = BTC['date'].dt.strftime('%d/%m/%Y')

LTC['date'] = pd.to_datetime(LTC['date'])
LTC['date'] = LTC['date'].dt.strftime('%d/%m/%Y')

ETH['date'] = pd.to_datetime(ETH['date'])
ETH['date'] = ETH['date'].dt.strftime('%d/%m/%Y')

# XRP["Date"] = pd.to_datetime(XRP["date"],infer_datetime_format=True)
# XRP['Date'] = XRP['Date'].dt.normalize()

XRP = XRP.rename(columns={'Date':'date'})
XRP['date'] = pd.to_datetime(XRP["date"],infer_datetime_format=True)
XRP['date'] = XRP['date'].dt.normalize()
XRP['date'] = XRP['date'].dt.strftime('%d/%m/%Y')

DOGE = DOGE.rename(columns={'Date':'date'})
DOGE["date"] = pd.to_datetime(DOGE["date"],infer_datetime_format=True)
DOGE['date'] = DOGE['date'].dt.normalize()
DOGE['date'] = DOGE['date'].dt.strftime('%d/%m/%Y')

# Drop 'Unnamed:0' column: 
BTC = BTC.drop('Unnamed: 0', axis=1)
LTC = LTC.drop('Unnamed: 0', axis=1)
ETH = ETH.drop('Unnamed: 0', axis=1)
XRP = XRP.drop('Unnamed: 0', axis=1)
DOGE = DOGE.drop('Unnamed: 0', axis=1)

# Set date column as index to merge and have the same days of trade: 
BTC = BTC.set_index('date')
LTC = LTC.set_index('date')
ETH = ETH.set_index('date')
XRP = XRP.set_index('date')
DOGE= DOGE.set_index('date')

crypto_list = [BTC,LTC,ETH, XRP,DOGE]

In [59]:
# We concat all cryptos in 1 Df
cryptos = pd.concat(crypto_list,axis=1,join="inner",ignore_index=False,sort=False, copy=False)

Merge all in a main DF so i have the matchin days of trade 

In [60]:
final_df = pd.merge(cryptos,main_,how="inner",on='date')

Now that we have all the data for the sam trading days ,we can splpit the DF by columns for further analysis:

In [61]:
# Seprarating columns 
crypto_col = ['btc_p', 'ltc_p', 'eth_p', 'xrp_p', 'doge_p']
currency_col = ['usd_gbp', 'usd_gbp_ %','usd_eur', 'usd_eur_%', 'usd_yuan', 'usd_yuan_%', 'usd_yen','usd_yen_ %'] 
comm_col = ['wti_oil_pprice', 'brent_oil_p', 'gold_p', 'silver_p','steel_price']
rates_col = ['LIBOR', 'eonia', 'fed_rate']

In [62]:
# Slicing the DF 
crypto_series = final_df[crypto_col]
currency_series = final_df[currency_col]
commodity_series = final_df[comm_col]
rates_series = final_df[rates_col]

In [63]:
# Saving to cvs for further analysis
final_df("../final_data/final_df.csv")
crypto_series("../final_data/crypto_series.csv")
currency_series("../final_data/currency_series.csv")
commodity_series("../final_data/commodity_series.csv")
rates_series("../final_data/rates_series.csv")

TypeError: 'DataFrame' object is not callable